Esto es para la versión de jupyter, permitimos que nos muestre los warnings de compilación que tenga un bloque de código. Tambíen indicamos que nos muestre los warnings por código inalcanzable. Pero shhh, son spoilers del post.

In [1]:
println("start")

start


In [2]:
interp.configureCompiler(_.settings.nowarn.value = false)

In [3]:
interp.configureCompiler(_.settings.warnDeadCode.value = true)

## ¿Que es el pattern matching?

Una estructura de control, pensada para comprobar si un elemento cumple ciertas condiciones. Si no la conocias antes es similar en sintaxis a un switch, pero nos permite mayor precisión.
Para aplicarlo solo necesitamos poner a continuación de el elemento sobre el que queremos aplicarlo la plabra reservada match e indicar cada uno de los casos que nos interesan.

Comencemos con unos ejemplos sencillos:

In [4]:
val stringExample = "hola"
stringExample match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case _ => "???"
}

stringExample: String = "hola"
res3_1: String = "saludos"

Vamos a describir que está ocurriendo aquí. tenemos nuestro valor asignado y empezamos el matcheado, en este caso, queremos comprobar si el valor a procesar es igual al string "hola", en caso de ser correcto, se ejecutará el codigo de su parte derecha, si no pasará al siguiente caso y repetirá lo mismo. Podeis ver que el último caso, se representa solo con `_` esta es la forma en scala de indicar "cualquier otro caso", por lo que si ninguno de los anteriores ha sido satisfactorio, en este caso siempre ejecutará su código.

Además aquí va la primera diferencia, en el patter matching, a diferencia de un switch, unicamente ejecutará el primer trozo de código que satisfaga la condición, por lo que el orden que indiquemos los casos importa.

In [5]:
val stringExample = "hola"
stringExample match {
    case _ => "???"
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
}

cmd4.sc:3: patterns after a variable pattern cannot match (SLS 8.1.1)
    case _ => "???"
         ^cmd4.sc:4: unreachable code due to variable pattern on line 3
    case "hola" => "saludos"
                   ^cmd4.sc:5: unreachable code due to variable pattern on line 3
    case "adios" => "hasta pronto"
                    ^cmd4.sc:4: unreachable code
    case "hola" => "saludos"
                   ^

stringExample: String = "hola"
res4_1: String = "???"

En este caso, pusimos en primer caso el comodin, por lo que cualquier valor al ser comparado entrará, por lo que el resto de casos son inaccesibles. Además, podemos ver un error de compilación `patterns after a variable pattern cannot match` que nos indica que tenemos código inalcanzable, todos los casos que hay tras el `case _` se podría borrar, lo que nos da una pista que nuestro pattern matching podría estar mal, o que podemos prescindir de casos inalcanzables.

## Pero esto es solo la punta del iceberg.
### Comparar con múltiples casos.

En pattern matching no solo podemos hacer condiciones de igualdad, como hemos visto, si no que podemos hacer una variada cantidad de acciones, la primera que vamos a ver, es en caso de tener multiples valores que pueden satisfacer un mismo caso:

In [6]:
val stringExample1 = "hola"
stringExample1 match {
    case "hola" | "holi" => "saludos"
    case "adios" => "hasta pronto"
    case _ => "???"
}

val stringExample2 = "holi"
stringExample2 match {
    case "hola" | "holi" => "saludos"
    case "adios" => "hasta pronto"
    case _ => "???"
}

stringExample1: String = "hola"
res5_1: String = "saludos"
stringExample2: String = "holi"
res5_3: String = "saludos"

## Asignación a un valor

Otra de ellas es la de la asignación a un valor, para poder procesarla si cumple una condición.

In [7]:
val stringExample = "ya estoy"
stringExample match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case x => s"el valor $x no está contemplado"
}

stringExample: String = "ya estoy"
res6_1: String = "el valor ya estoy no est\u00e1 contemplado"

podemos ver, que en este caso hemos cambiado nuestro comodin `_` por `x`, realmente el comodín no indica "en cualquier otro caso" realmente es una asignación a un valor que no podemos usar, es decir, indicarle que el valor no nos interesaba en ese momento. Pero ahora si que la hemos asignado a x, y podemos garantizar que nunca contendrá los valores "hola" y "adios", y que solo podrá usarse el valor x.

Aquí quiero hacer otro inciso, y es el caso de nombrar el valor donde se asignará el valor, ya que scala tiene unas reglas.

Pongamos el siguiente ejemplo, en el que asignamos el valor a un nombre ya existente.

In [8]:
val x = "soy x"

val stringExample = "ya estoy"
stringExample match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case x => s"el valor $x no está contemplado"
}

x: String = "soy x"
stringExample: String = "ya estoy"
res7_2: String = "el valor ya estoy no est\u00e1 contemplado"

En este caso, el valor `x` de dentro del `match` impedirá en el contexto de la derecha que se pueda acceder al valor `x` externo, esto se llama ocultamiento de valor, o "variable shadowing", y puede llevar a confusión en algunos casos, por suerte, el compilador de scala, tambien nos dará una advertencia en caso de que ocurra esto.

¿Y si yo quisiera comparar un caso, con el contenido de un valor que tengo definido? En el pattern matching se puede, este no obliga a tener que declarar la definición de cada paso, si no que podemos hacerla programatica, pero siguiendo unas reglas, ya que, como hemos visto, un valor con el que nos gustaría comparar, puede combertirse en una nueva asignación. Entonces ¿como podría crear un caso si es igual a mi valor externo `x`? Indicando que este nombre de valor no es para asignarlo, si no compararlo, rodeando el valor con comillas `` `x` ``:

In [9]:
val x = "soy x"

val stringExample = "soy x"
stringExample match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case `x` => s"es el valor que tenía en x"
    case _ => "ninguno de los anteriores"
}

x: String = "soy x"
stringExample: String = "soy x"
res8_2: String = "es el valor que ten\u00eda en x"

Y si no te fias que esto sea así, pongamos un ejemplo que no coincida.

In [10]:
val x = "soy x"

val stringExample2 = "no soy x"
stringExample2 match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case `x` => s"es el valor que tenía en x"
    case _ => "ninguno de los anteriores"
}

x: String = "soy x"
stringExample2: String = "no soy x"
res9_2: String = "ninguno de los anteriores"

Otra forma más sencilla, es seguir [la guia de estilo de scala](https://docs.scala-lang.org/style/naming-conventions.html#constants-values-variable-and-methods), en la que indica que los valores definidos, han de empezar con mayuscula, lo que reconoce que no es un valor a asignar

In [11]:
val X = "soy x"

val stringExample = "soy x"
stringExample match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case X => s"es el valor que tenía en x"
    case _ => "ninguno de los anteriores"
}

X: String = "soy x"
stringExample: String = "soy x"
res10_2: String = "es el valor que ten\u00eda en x"

In [12]:
val X = "soy x"

val stringExample2 = "no soy x"
stringExample2 match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case X => s"es el valor que tenía en x"
    case _ => "ninguno de los anteriores"
}

X: String = "soy x"
stringExample2: String = "no soy x"
res11_2: String = "ninguno de los anteriores"

### Refinar la condición de nuestro caso.

Genial, ahora que podemos asignar el valor, podemos llegar a otra de las ventajas del pattern matching, y es el poder refinar la condición sin tener que ser siempre por igualdad, como hicimos hasta el momento. Pongamos el ejemplo que quisieramos tratar los strings que comiencen por `h` de manera distinta, excepto el caso que tenemos ya, contemplado con la palabra , con los conocimientos que tenemos actualmente, nuestro código quedaría algo tal que así:

In [13]:
val stringExample2 = "habana"
stringExample2 match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case x => if (x.head == 'h') "comienza por h" else "ninguno de los anteriores"
}

stringExample2: String = "habana"
res12_1: String = "comienza por h"

pero la utilidad del pattern matching es la de aplanar todos los posibles casos, y no empezar a anidar posibles casos más complejos, por lo que podemos hacer uso de la asignación del valor, y hacer filtrado de casos de la siguiente manera.

In [14]:
val stringExample2 = "habana"
stringExample2 match {
    case "hola" => "saludos"
    case "adios" => "hasta pronto"
    case x if x.head == 'h' => "comienza por h"
    case _ => "ninguno de los anteriores"
}

stringExample2: String = "habana"
res13_1: String = "comienza por h"

De esta manera, vemos más claramente los 4 posibles casos, con su condición por delante, y no tener que rebuscar lógica escondida. Lo único a tener en cuenta, es que este `if` no necesita parentesis en la condición a diferencia de la estructura de control clásica en scala.

### Comprobación del tipo del elemento matcheado.

¿Hasta ahora todo bien? Comencemos a trabajar con más tipos aparte de nuestro querido String, y empecemos a ver casos en los que mezclamos tipos, ¿Como podríamos sacar un caso si es string otro si es Integer, y un último para el resto? En principio sería algo tal que así


In [15]:
def matchfun(x: Any): String =
 x match {
     case x if x.isInstanceOf[String] =>
       val xString = x.asInstanceOf[String]
       s"tengo el string $xString"
     case x if x.isInstanceOf[Int] =>
       val xInt = x.asInstanceOf[Int]
       s"tengo el integer $xInt"
     case _ => "es otro tipo"
 }

defined function matchfun

In [16]:
matchfun("hola")
matchfun(42)
matchfun(12.4)

res15_0: String = "tengo el string hola"
res15_1: String = "tengo el integer 42"
res15_2: String = "es otro tipo"

Esta forma genera mucho código repetitivo, por lo que tenemos una forma más concisa de describirlo, no solo saber si es de un tipo, además automaticamente hace el cambio de tipo para que trabajemos en nuestro contexto.

In [17]:
def matchfun2(x: Any): String =
 x match {
     case x: String => s"tengo el string $x"
     case x: Int => s"tengo el integer $x"
     case _ => "es otro tipo"
 }

defined function matchfun2

In [18]:
matchfun2("hola")
matchfun2(42)
matchfun2(12.4)

res17_0: String = "tengo el string hola"
res17_1: String = "tengo el integer 42"
res17_2: String = "es otro tipo"

Esta forma es mucho más concisa y nos evita mancharnos las manos con funciones como `asInstanceOf`.

#### Cuidado con las comprobaciones de algunas clases

Una cosa que tenemos que tener en cuenta, es que estas comprobaciones se hacen en tiempo de ejecución, y la JVM tiene una limitación, y es con el úso de tipos parametricos. Esto lleva a que si quisieramos comparar y saber si es de tipo `List[String]` o `List[Int]` no podríamos facilmente, ya que la información de que tipo de elemento contiene, durante la ejecución se pierde:

In [19]:
def matchfun2(x: List[Any]): String =
 x match {
     case x: List[String] => s"tengo una lista de string de longitud ${x.size}"
     case x: List[Int] => s"tengo una lista de integer de longitud ${x.size}"
     case _ => "es otro tipo de lista"
 }

cmd18.sc:3: non-variable type argument String in type pattern List[String] (the underlying of List[String]) is unchecked since it is eliminated by erasure
     case x: List[String] => s"tengo una lista de string de longitud ${x.size}"
             ^cmd18.sc:4: non-variable type argument Int in type pattern List[Int] (the underlying of List[Int]) is unchecked since it is eliminated by erasure
     case x: List[Int] => s"tengo una lista de integer de longitud ${x.size}"
             ^cmd18.sc:4: unreachable code
     case x: List[Int] => s"tengo una lista de integer de longitud ${x.size}"
                          ^

defined function matchfun2

In [20]:
matchfun2(List("hola", "adios"))
matchfun2(List(42))
matchfun2(List(4.2f))

res19_0: String = "tengo una lista de string de longitud 2"
res19_1: String = "tengo una lista de string de longitud 1"
res19_2: String = "tengo una lista de string de longitud 1"

### ADT's en pattern matching

Ya que hemos visto que scala permite comprobar el tipo del elemento, para poder realizar una acción para cada tipo. Es por esto que quiero pararme a comentar una particularidad la programación funcional, que por supuesto se aplica en scala, y es el uso de los Tipos Algebraicos de Datos, Algebraic Data Types en inglés o ADT para que sea más corto. Esto es una representación de los datos que se basa en el producto y suma de tipos, por ejemplo un producto de String e integer sería la siguiente `case class`

In [21]:
case class Usuario(nombre: String, edad: Int)

defined class Usuario

Y una  suma de tipos se puede representar de múltiples maneras, pero la más común es el uso de `sealed trait` por ejemplo.

In [22]:
sealed trait Trabajador
case class Currito(nombre: String) extends Trabajador
case class Jefe(nombre: String, subordinados: List[Trabajador]) extends Trabajador

defined trait Trabajador
defined class Currito
defined class Jefe

En este vemos una representación de que sería un trabajador, o es alguien con subordinados a su cargo, o es un currante sin nadie a su cargo. Al ser un `sealed trait` solo permite estas dos posiblidiades de tipo de trabajador, y no se puede extender en ningún otro lado. Esta forma de representación de datos es muy usada en scala, incluso en elementos que nos da el lenguaje, como sería Option, que tiene dos posibles elementos, Some, que indica que contiene un elemento, o None, que no contiene ninguno.

Dada la particularidad de esta suma de tipos, el pattern matching suele ser una herramienta muy común y util para poder actuar segun el tipo de dato que podamos encontrarnos.

In [23]:
def quienEs(t: Trabajador): String =
t match {
    case j: Jefe => s"${j.nombre} es jefe de ${j.subordinados.size} empleados"
    case c: Currito => s"${c.nombre} es un gran trabajador"
}

def tengoDato(o: Option[Int]): String =
o match {
    case s: Some[Int] => s"tenemos el valor ${s.get}"
    case None => "no tenemos valor" // en este caso, no comparamos por tipo, si no contra el objeto único que representa un Option vacío
}

defined function quienEs
defined function tengoDato

In [24]:
quienEs(Jefe("JM", List(Currito("Ar"), Currito("J"))))
quienEs(Currito("Ar"))

tengoDato(Some(23))
tengoDato(None)

res23_0: String = "JM es jefe de 2 empleados"
res23_1: String = "Ar es un gran trabajador"
res23_2: String = "tenemos el valor 23"
res23_3: String = "no tenemos valor"

### Extractores

Como puedes comprobar, el patter matching no puede ver más allá de que es una lista, y el tipo contenido nunca lo tiene en cuenta, por lo que siempre entrará en el primer caso. Se podría comprobar que tipo de elemento contiene en la cabeza, pero siempre tedremos el problema en listas vacías, ya que nos resultará imposible poder comprobarlo. Eso si, como todo posible punto de error, el compilador nos informará para que lo tengamos en cuenta con el siguiente warning `List[String] is unchecked since it is eliminated by erasure`, o traducido, "List[String] no está chequeado porque ha sido borrado" ya que el compilador lo traducirá a `case x: List => ...` borrando el tipo de la lista. Como os podreis imaginar, esto no ocurre solo con listas, si no con todos los tipos parametricos.

Pero no nos quedemos solo con las limitaciones, porque al fin llega uno de los elementos más potentes del pattern matching, y mi favorito, los extractores. Pongamos que ya somos mayorcitos y no trabajamos solo con tipos simples como String, Int, etc, si no que ya tenemos estructuras más complejas, por ejemplo una tupla, en la que nos gustaría hacer varios casos, segun el contenido de esta, como hemos hecho hasta ahora, con nuestro conocimiento actual, podríamos hacer algo tal que así:

In [25]:
def tuplaMatch(x: (String, Int)): String =
 x match {
     case x if x._1 == "hola" & x._2  == 10 => "hola con valor igual que 10"
     case x if x._1 == "hola" & x._2 > 10 => "hola con valor mayor que 10"
     case x if x._1 == "hola" => "hola con valor menor a 10"
     case x => s"la palabra es ${x._1} con valor ${x._2}"
 }

defined function tuplaMatch

In [26]:
tuplaMatch(("hola", 10))
tuplaMatch(("hola", 42))
tuplaMatch(("hola", 2))
tuplaMatch(("adios", 42))

res25_0: String = "hola con valor igual que 10"
res25_1: String = "hola con valor mayor que 10"
res25_2: String = "hola con valor menor a 10"
res25_3: String = "la palabra es adios con valor 42"

El código es correcto, pero hasta el momento la ventaja del pattern matching principal es una descripción muy gráfica de la lógica, y aquí es donde entra el uso de los extractores. Con estos, podemos comprobar o asignar los elementos internos de una manera mucho más parecido a la representación de la construcción de la clase.

Por ejemplo, para crear una tupla, la forma que lo hacemos es poniendo los elementos necesarios entre parentesis, y separados por una coma, en este caso a ser una tupla de dos elementos, se podría representar tal que así:

In [27]:
val tupla: (String, Int) = ("texto", 1)

tupla: (String, Int) = ("texto", 1)

In [28]:
def tuplaMatch2(x: (String, Int)): String =
 x match {
     case ("hola", 10) => "hola con valor igual que 10"
     case ("hola", x2) if x2 > 10 => "hola con valor mayor que 10"
     case ("hola", _) => "hola con valor menor a 10"
     case (x1, x2) => s"la palabra es ${x1} con valor ${x2}"
 }

defined function tuplaMatch2

In [29]:
tuplaMatch(("hola", 10))
tuplaMatch(("hola", 42))
tuplaMatch(("hola", 2))
tuplaMatch(("adios", 42))

res28_0: String = "hola con valor igual que 10"
res28_1: String = "hola con valor mayor que 10"
res28_2: String = "hola con valor menor a 10"
res28_3: String = "la palabra es adios con valor 42"

Hay que tener en cuenta, que el uso de extractores no es solo para pattern matching, también se pueden  usar en las asignaciones

In [30]:
val tupla: (String, Int) = ("texto", 1)
val (primero, segundo) = tupla

tupla: (String, Int) = ("texto", 1)
primero: String = "texto"
segundo: Int = 1

Los extractores no solo permiten comparar por igualdad, o asignar los elementos internos, también permite comprobar el tipo de los elementos internos.

In [31]:
def tuplaAnyMatch(x: (Any, Any)): String =
  x match {
      case (x: String, y: String) => s"dos strings primero: $x segundo: $y"
      case (x: String, _) => "solo el primero es string: $x"
      case (_, x: String) => "solo el segundo es string: $x"
      case _ => "ninguno es string"
  }

defined function tuplaAnyMatch

In [32]:
tuplaAnyMatch(("hola", "adios"))
tuplaAnyMatch(("hola", 42))
tuplaAnyMatch((1, "adios"))
tuplaAnyMatch((1, 42))

res31_0: String = "dos strings primero: hola segundo: adios"
res31_1: String = "solo el primero es string: $x"
res31_2: String = "solo el segundo es string: $x"
res31_3: String = "ninguno es string"

#### Bricomanía: crea tus propios extractores

¿Y como puede ser esto posible? ¿Cuando se que algo se puede descomponer o no? Muy sencillo, tenemos que ver si existe un método en esa clase llamado `unapply`, este es el truco que usa scala para poder descomponer algo. Para los tipos tipicos de scala, como tuplas, listas, o toda case class que creamos, scala ya tiene preparado este método para nosotros en el companion object, pero si por la razón que sea, no tiene este método, podemos crearlo nosotros.

##### Extractores básicos

Lo primero a tener en cuenta es los elementos que entran en el método, en este caso siempre tiene que ser uno, y del tipo que queremos descomponer, y lo que devolverá, siempre ha de ser un Option, que será del tipo extraido.

Veamos un ejeplo primero, en el que queremos comprobar si un string se puede transformar a un integer. El problema es que el método toInteger que scala nos provee, lanza excepciones, por lo que no es seguro usarlo desde un pattern matching, pero nosotros podemos crar una clase que lo permita. 

In [33]:
object ValidIntString { // creamos el objeto que permitirá extraer el string si es valido
    def unapply(string: String): Option[Int] = // esperamos descomponer un string, y poder sacar un integer si es posible
    try {
        Some(string.toInt) // Si logra ejecutar sin excepciones, devolverá un some con el valor
    } catch {
        case _ : Throwable => None // en caso que no fuera integer, lanzaría excepción, por lo que devolvemos None
    }
}

defined object ValidIntString

Ahora podemos usar nuestro flamante nuevo extractor

In [34]:
"123" match {
    case ValidIntString(n) => s"es un integer con valor $n"
    case _ => "no es integer"
}

"hola" match {
    case ValidIntString(n) => s"es un integer con valor $n"
    case _ => "no es integer"
}

res33_0: String = "es un integer con valor 123"
res33_1: String = "no es integer"

Funciona perfectamente, pero y si quisiera devolver más de un elemento, como por ejemplo hacen en la tupla que vimos antes? Tenemos que devolver una tupla simplemente. Por ejemplo, queremos devolver el valor si se podía transformar a integer, y el doble de este valor en una tupla.

In [35]:
object ValidIntStringWithDouble { // creamos el objeto que permitirá extraer el string si es valido
    def unapply(string: String): Option[(Int, Int)] = // esperamos descomponer un string, y poder sacar una tupla de integes si es posible
    try {
        val x = string.toInt
        Some(x, x * 2) // Si logra ejecutar sin excepciones, devolverá un Some con el valor
    } catch {
        case _: Throwable => None // en caso que no fuera integer, lanzaría excepción, por lo que devolvemos None
    }
}

defined object ValidIntStringWithDouble

In [36]:
"123" match {
    case ValidIntStringWithDouble(n, n2) => s"es un integer con valor $n y su doble $n2"
    case _ => "no es integer"
}

"hola" match {
    case ValidIntStringWithDouble(n, n2) => s"es un integer con valor $n y su doble $n2"
    case _ => "no es integer"
}

res35_0: String = "es un integer con valor 123 y su doble 246"
res35_1: String = "no es integer"

##### Extractores variadricos

O si necesitamos un número indeterminado de elementos a devolver, podemos hacer de la variante variadrica unapplySeq, en el que devolveremos una secuencia de elementos. Cuando se hace la extracción en el match, se tiene en cuenta el número de elementos que se le pasan como argumento.

In [37]:
object Split3Decimals { // creamos el objeto que permitirá extraer el string si es valido
    def unapplySeq(string: String): Option[List[Int]] = // esperamos descomponer un string, y devolver un número indefinido de parámetros.
    try {
        val x = string.toFloat
        val hasDecimals = x % 1 != 0 // si es par tendrá 2 elementos la lista, si es impar solo uno
        if (hasDecimals)
          Some(List((x / 1).toInt, (x % 1 * 1000000).toInt)) // Al ser par devolvemos 2 elementos
        else
          Some(List((x / 1).toInt)) // Al ser impar devolvemos solo uno
    } catch {
        case _: Throwable => None // en caso que no fuera integer, lanzaría excepción, por lo que devolvemos None
    }
}

defined object Split3Decimals

In [38]:
"123.0000" match {
    case Split3Decimals(n1, n2) => s"tiene decimales: $n1 . $n2"
    case Split3Decimals(n1) => s"es entero y tenemos $n1 solo"
    case _ => "no es numerico"
}

"123.56454" match {
    case Split3Decimals(n1, n2) => s"tiene decimales: $n1 . $n2"
    case Split3Decimals(n1) => s"es entero y tenemos $n1 solo"
    case _ => "no es numerico"
}

"hola" match {
    case Split3Decimals(n1, n2) => s"tiene decimales: $n1 y $n2"
    case Split3Decimals(n1) => s"es entero y tenemos $n1 solo"
    case _ => "no es numerico"
}

res37_0: String = "es entero y tenemos 123 solo"
res37_1: String = "tiene decimales: 123 . 564537"
res37_2: String = "no es numerico"

#### Extractores Booleanos

Por último, scala permite otro tipo de extractor, en el que no interesa extraer un elemento, si no si cumple una propiedad. Al igual que hacemos en la parte de refinado, en el que podemos comprobar si cumple una condición, declarandolo explicitamente, tambien podríamos encapsular esa lógica para darle un nombre legible. Para hacer esto, tambien tenemos que crear un extractor con el método unapply, pero en vez de devolver un `Option`, solo tenemos que devolver un Booleano

In [39]:
object IsEaven {
    def unapply(int: Int): Boolean = int % 2 == 0
}

defined object IsEaven

In [40]:
54 match {
    case IsEaven() => "el valor es par"
    case _ => "el valor es impar"
}

45 match {
    case IsEaven() => "el valor es par"
    case _ => "el valor es impar"
}

res39_0: String = "el valor es par"
res39_1: String = "el valor es impar"

#### Usos de extractores ya implementados en scala

Con estos ejemplos, podemos ver que los extractores no solo sirven para facilitarnos acceder a los elementos, si no que también nos permiten hacer validaciones. Por ejemplo, en scala se usa para permitir el uso de regex en pattern matching, y poder extraer los elementos (o grupos) que capturamos.

In [41]:
val date = raw"(\d{4})-(\d{2})-(\d{2})".r

"2004-01-20" match {
    case date(year, month, day) => s"año: $year, mes: $month, dia: $day"
    case _ => "no es una fecha"
}

"hola" match {
    case date(year, month, day) => s"año: $year, mes: $month, dia: $day"
    case _ => "no es una fecha"
}

date: scala.util.matching.Regex = (\d{4})-(\d{2})-(\d{2})
res40_1: String = "a\u00f1o: 2004, mes: 01, dia: 20"
res40_2: String = "no es una fecha"

Otro caso es poder matchera las listas, esperando un número especifico de elementos

In [42]:
List(1, 2, 3) match {
    case List(n1) => s"tiene solo un elemento $n1"
    case List(n1, n2) => s"tiene dos elementos $n1, $n2"
    case List(n1, n2, n3) => s"tiene tres elementos $n1, $n2, $n3"
    case l => s"tiene demasiados elementos, exactamente  ${l.size}"
}

List(1) match {
    case List(n1) => s"tiene solo un elemento $n1"
    case List(n1, n2) => s"tiene dos elementos $n1, $n2"
    case List(n1, n2, n3) => s"tiene tres elementos $n1, $n2, $n3"
    case l => s"tiene demasiados elementos, exactamente  ${l.size}"
}

List(1, 2, 3, 4) match {
    case List(n1) => s"tiene solo un elemento $n1"
    case List(n1, n2) => s"tiene dos elementos $n1, $n2"
    case List(n1, n2, n3) => s"tiene tres elementos $n1, $n2, $n3"
    case l => s"tiene demasiados elementos, exactamente  ${l.size}"
}

res41_0: String = "tiene tres elementos 1, 2, 3"
res41_1: String = "tiene solo un elemento 1"
res41_2: String = "tiene demasiados elementos, exactamente  4"

### Obtener el elemento original y poder aplicar un patrón.

El uso de extractores es muy común, pero podemos llegar al caso que nos interesaría poder tener el valor original en la condición, por lo que podemos hacer uso del símbolo `@` con el que podemos asignar el valor original a un valor, y a continuación del símbolo `@` descomponerlo con un patrón.

In [43]:
val date = raw"(\d{4})-(\d{2})-(\d{2})".r

"2004-01-20" match {
  case d @ date(year, month, day) => s"año: $year, mes: $month, dia: $day original $d"
  case _ => "no es una fecha"
}

date: scala.util.matching.Regex = (\d{4})-(\d{2})-(\d{2})
res42_1: String = "a\u00f1o: 2004, mes: 01, dia: 20 original 2004-01-20"

## Ahora todo a la vez!

Haciendo uso de todo lo visto hasta ahora se puede realizar comparativas complejas en muy poco código:

In [44]:
val date = raw"(\d{4})-(\d{2})-(\d{2})".r // regex para fechas con guion 2020-01-01

val year19xx = raw"19(\d{2})".r // regex para números de 4 cifras que comienzan por 19xx y extrae el xx

val specialYear = "2001"

def whatDayIs(dateStr: String): String =
dateStr match {
    // comparación con un valor tras la extracción
    case date(`specialYear`, _, _) => "mi año especial :D"
    // comparación con literal tras extracción
    case date(_, "01", "01") => s"feliz año nuevo!"
    // asignación del valor original y comparación en la extracción
    case d @ date(_, "02", "29") => s"es año bisiesto $d"
    // varios posibles casos de un elemento extraido
    case date("1800" | "1700", _, _) => "eso es muy viejo"
    // refinamiento tras extracción
    case date(year, month, day) if year.reverse == (month + day) => s"la fecha es capicua $year$month$day"
    // extracción de un elemento obtenido por una extracción
    case date(year19xx(year19), month, day) => s"$day del $month del año $year19"
    case date(year, month, day) => s"año: $year, mes: $month, dia: $day"
    case _ => "no es una fecha"
}

date: scala.util.matching.Regex = (\d{4})-(\d{2})-(\d{2})
year19xx: scala.util.matching.Regex = 19(\d{2})
specialYear: String = "2001"
defined function whatDayIs

In [45]:
whatDayIs("2001-03-01")
whatDayIs("2021-01-01")
whatDayIs("2020-02-29")
whatDayIs("1800-03-29")
whatDayIs("1700-03-29")
whatDayIs("2020-02-02")
whatDayIs("1995-02-03")
whatDayIs("2021-31-10")
whatDayIs("2021")

res44_0: String = "mi a\u00f1o especial :D"
res44_1: String = "feliz a\u00f1o nuevo!"
res44_2: String = "es a\u00f1o bisiesto 2020-02-29"
res44_3: String = "eso es muy viejo"
res44_4: String = "eso es muy viejo"
res44_5: String = "la fecha es capicua 20200202"
res44_6: String = "03 del 02 del a\u00f1o 95"
res44_7: String = "a\u00f1o: 2021, mes: 31, dia: 10"
res44_8: String = "no es una fecha"

### Un error que todos cometemos

Ya vimos al comienzo que el compilador nos daba un mensaje de advertencia cuando teníamos casos que no eran alcanzables, pero planteo otra duda. ¿Que ocurre si tenemos un caso que no está contemplado?

In [46]:
def tengoDato(o: Option[Int]): String =
o match {
    case Some(0) => s"tenemos el valor 0" // solo contemplamos Some con el valor 0
    case None => "no tenemos valor"
}

cmd45.sc:2: match may not be exhaustive.
It would fail on the following input: Some((x: Int forSome x not in 0))
o match {
^

defined function tengoDato

Ya vemos en este caso que solo con la definición ya nos advierte el compilador de que algo falta. Pero si aun así hacemos caso omiso, al ejecutar:

In [47]:
tengoDato(Some(1))

: 

Obtenemos un error en la ejecución de tipo `scala.MatchError`, y ya hemos dicho que esto en scala, hay que evitarlo siempre que sea posible.

Como bien sabrás, scala está muy orientado a que se programe segun el paradigma funcional, lo que nos lleva a las funciones puras. Si no conocías el concepto de función pura, podemos resumirlo como que a todo elemento que entra en una función, tiene que devolver un valor, pero una excepción no es un valor, o no al menos uno que se pueda recoger solo haciendo una asignación.

Hay que tener en cuenta que esta comprobación de exhausividad solo funciona si trabajamos con ADT's, si realizamos este match en un tipo primitivo como son `String`, o `Int`, el compilador no va a poder decirnos estas advertencias.

In [48]:
def noExhaustivo(o: Int): String =
o match {
    case 0 => s"tenemos el valor 0"
    case 1 => s"tenemos el valor 1"
}

defined function noExhaustivo

In [49]:
noExhaustivo(0)
noExhaustivo(1)
noExhaustivo(2)

: 

Por lo que es siempre recomendable poner un caso por defecto (`case _ => `) que lo evite si estamos haciendo match sobre un elemento primitivo, o clases que no sean ADTs.

### Un tip si eres nuevo (o no te fias ni de ti mismo)

El compilador de scala sabe que un pattern matching es un posible foco de excepciones, por lo que en muchos casos, si ve que no se contemplan todos los casos de entrada, o lo que es lo mismo, no es exhaustivo nos dará un advertencia a nivel de warning. 

Yo te doy un consejo, es una buena practica, hacer que una build falle si hay algun mensaje, solo tienes que añadir la siguiente opción en tu proyecto sbt:

```scala
scalacOptions += "-Xfatal-warnings"
```

o si eres de los que trabaja con maven, en el plugin de scala añade junto a tus opciones:

```xml
<executions>
  <execution>
    <configuration>
      <args>  
        <arg>-Xfatal-warnings</arg>
      </args>
    </configuration>
  </execution>
</executions>
```


Con esto, ya tenemos todas las herramientas necesarias para convertirnos en un ninja del patter matching, pudiendo hacer una gran y compleja lógica de una manera muy legible y mantenible, y teniendo al compilador como red de seguridad que nos supervise.

## Como reducir más nuestro código

Aunque ya vimos al inicio como podemos indicar un patter matching sobre un valor, si que es bueno conocer un elemento de azucar sintactico que scala nos da. Muchas veces cuando queremos hacer un pattern matching es creando una lambda, por ejemplo en un método map.

In [50]:
val optval = Some(4)

optval.map(x => x match {
    case 1 => "es 1"
    case 2 => "es el número 2"
    case _ => "es otro número"
})

optval: Some[Int] = Some(4)
res49_1: Option[String] = Some("es otro n\u00famero")

Si vemos que nuestra lambda se puede representar solo con el pattern matching, scala nos permite evitar el inicio `x => x match {` cambiando los parentesis entre los que se va a implementar, por las llaves del patter matching directamente.

In [51]:
val optval = Some(4)

optval.map{
    case 1 => "es 1"
    case 2 => "es el número 2"
    case _ => "es otro número"
}

optval: Some[Int] = Some(4)
res50_1: Option[String] = Some("es otro n\u00famero")

## Funciones parciales

{...}

## Scala 3

(Nota: esto no funciona con jupyter)

Ahora toca mirar al futuro proximo, en pocos meses de la fecha de este post, saldrá una nueva versión de scala, denominada dotty o scala 3. En esta se ha reescrito el compilador y va a tener muchas novedades. Respecto al tema del pattern matching, no va a tener grandes cambios, todo lo que se ha visto para indicar la condición, se mantiene tal cual.

### Match como 'función'
Pero si que hay un cambio con respecto a la palabra match, sigue siendo una palabra reservada, pero ahora se puede usar como llamada a un método, es decir, usando punto respecto al valor.

In [31]:
45.match{
    case 1 => "es 1"
    case 2 => "es el número 2"
    case _ => "es otro número"
}

(console):1:4 expected id
45.match{
   ^

: 

Esta forma trata de permitirnos cambiar la prioridad para procesar el pattern matching, permitiendonos integrarlo facilmente con otros elementos, por ejemplo:

In [42]:
if 4.match:
     case 5 => true
     case _ => false
then
  "valor 5"
else
  "otro valor"

(console):1:4 expected "("
if 4.match:
   ^

: 

### Extractores 'irrefutables'

Otra de las mejoras que se tiene en scala 3, es en la creación de extractores. Una limitación que tienen actualmente en scala 2 los extractores, es la obligación de devolver los elementos extraidos en un Option, lo que representa que esa extracción puede no ir bien. Esto impide crear extractores, que sabemos que siempre irán correctamente, o como lo llaman en la documentación 'irrefutables', como por ejemplo el siguiente.

In [32]:
object NextNumber {
    def unapply(int: Int): Option[Int] = Some(int + 1)
}

5 match {
    case NextNumber(6) => "el valor es válido"
    case _ => "no es válido"
}

defined object NextNumber
res31_1: String = "el valor es v\u00e1lido"

Como se ve, en este extractor se devuelve un elemento `Option[Int]` pero nunca va a tener el caso que devolvamos `None`. Esto en scala 3 se ha mejorado, permitiendo el uso de extractores que no solo devuelvan `Option`, tambien acepta ahora `Product`, recordar que a este último pertenecen tuplas y todas las `case class`.

In [43]:
object NextNumber {
    def unapply(int: Int): Int = int + 1
}

defined object NextNumber

In [43]:
5 match {
    case NextNumber(6) => "el valor es válido"
    case _ => "no es válido"
}

cmd43.sc:2: an unapply result must have a member `def isEmpty: Boolean`
    case NextNumber(6) => "el valor es válido"
         ^Compilation Failed

: 

Este tipo de patrón irrefutable, será el único que se permita usar en las asignaciones fuera del patter matching.

In [43]:
object NextNumber {
    def unapply(int: Int): Int = int + 1
}

val NextNumber(n2) = 10

cmd43.sc:5: an unapply result must have a member `def isEmpty: Boolean`
val NextNumber(n2) = 10
    ^Compilation Failed

: 

Y como último apunte ya que hablamos de extractores, se permitirá su uso en los for compresion, pero eso es tema para otro post ;)